# Am-241 and Ba-133 FCCD combination

In [1]:
using JSON

# read parameters from json files
open("parameters.json") do f
    global j_dl = JSON.parse(f)
end
open("geometry.json") do f
    global j_geo = JSON.parse(f)
end

det_matrix = Dict(j_geo["gerdageometry"][string(i)]["name"] => string(i) for i ∈ 42:71)
detectors = collect(j_geo["gerdageometry"][string(i)]["name"] for i ∈ 42:71)
filter!(!s->occursin(r"GD02D", s),detectors)
print("*** data loaded")

*** data loaded

In [2]:
# get dl growth
function dl_growth(det)
    j_dl[det]["time-at-room-temperature-yr"]
end
# get dl value
function DL_all(det, method)
    try
        j_dl[det][string(method,"-fccd")]
    catch
        print("DL value not found for detector: ", det)  
    end
end
function DL(det, method)
    DL_all(det, method)["central"]
end
function DL_sigma(det, method)
    dla = DL_all(det, method)
    error  = dla["corrPos"] + dla["uncorrPos"]
    error += dla["corrNeg"] + dla["uncorrNeg"]
    return error/2
end
# retrieve detector geometry
function height(det)
    j_geo["gerdageometry"][det_matrix[det]]["height-mm"]
end
function radius(det)
    j_geo["gerdageometry"][det_matrix[det]]["radius-mm"]
end
function corner_height(det)
    j_geo["gerdageometry"][det_matrix[det]]["corner"]["height-mm"]
end
function corner_radius(det)
    j_geo["gerdageometry"][det_matrix[det]]["corner"]["radius-mm"]
end
function groove_depth(det)
    j_geo["gerdageometry"][det_matrix[det]]["groove"]["depth-mm"]
end
function groove_inner_radius(det)
    j_geo["gerdageometry"][det_matrix[det]]["groove"]["inner-radius-mm"]
end
function groove_outer_radius(det)
    j_geo["gerdageometry"][det_matrix[det]]["groove"]["outer-radius-mm"]
end
function has_taper(det)
    haskey(j_geo["gerdageometry"][det_matrix[det]], "corner")
end
print("*** getter functions defined")

*** getter functions defined

In [3]:
# cylinder volume
function cylinder_volume(h, r)
    return π * h * r*r
end
# truncated cone volume
function cut_cone_volume(h, r, R)
    return 1/3 * π * h * (r*r + r*R + R*R)
end
# groove volume
function groove_volume(d, r, R)
    return π * d * (R*R - r*r)
end
function groove_volume(det)
    d = groove_depth(det)
    r = groove_inner_radius(det)
    R = groove_outer_radius(det)
    groove_volume(d,r,R)
end
# volume of conical detector
function volume_taper(h, r, ch, cr)
    vol = cut_cone_volume(ch, cr, r)
    vol += cylinder_volume(h-ch, r)
    return vol
end
function volume_taper(det)
    h = height(det)
    r = radius(det)
    ch = corner_height(det)
    cr = corner_radius(det)
    vol = volume_taper(h, r, ch, cr)
    vol -= groove_volume(det)
    return vol
end
# volume normal detector
function volume_notaper(det)
    h = height(det)
    r = radius(det)
    vol = cylinder_volume(h, r)
    vol -= groove_volume(det)
    return vol
end
# calculate volume of detector in cm3
function volume(det)
    return has_taper(det) ? 0.001*volume_taper(det) : 0.001*volume_notaper(det)
end
print("*** total volume functions defined")

*** total volume functions defined

In [4]:
# active volume functions tapered
function corner_radius_shift(det)
    tth = (radius(det) - corner_radius(det)) / corner_height(det)
    th = atan(tth)
    return tth*(1/sin(th) - 1)
end
function corner_height_shift(det)
    tth = corner_height(det) / (radius(det) - corner_radius(det));
    th = atan(tth);
    return  tth*(1/sin(th) - 1);
end
function active_volume_taper(det, DL)
    h = height(det)-2*DL
    r = radius(det)-DL
    ch = corner_height(det) + corner_height_shift(det)*DL
    cr = corner_radius(det) - corner_radius_shift(det)*DL
    vol = volume_taper(h, r, ch, cr)
    vol += cylinder_volume(DL, groove_outer_radius(det))
    vol -= groove_volume(det)
    return vol
end
# active volume of normal detector
function active_volume_notaper(det, DL)
    h = height(det)-2*DL
    r = radius(det)-DL
    vol = cylinder_volume(h, r)
    vol += cylinder_volume(DL, groove_outer_radius(det))
    vol -= groove_volume(det)
    return vol
end
function active_volume(det, DL)
    return has_taper(det) ? 0.001*active_volume_taper(det, DL) : 0.001*active_volume_notaper(det, DL)
end
function active_volume_am(det)
    active_volume(det, DL(det,"am")["central"])
end
function active_volume_ba(det)
    active_volume(det, DL(det, "ba")["central"])
end
print("*** active volume functions defined\n")

*** active volume functions defined


In [5]:
using Formatting
function combined_DL(det)
    dl_c = 0
    w = 0
    uc_pos = 0
    uc_neg = 0
    for m ∈ ["am","ba"]
        dla = DL_all(det, m)
        s_i = DL_sigma(det, m)
        w_i = 1/(s_i*s_i)
        dl_c += w_i * dla["central"]
        w += w_i
        uc_pos += dla["uncorrPos"]*dla["uncorrPos"]
        uc_neg += dla["uncorrNeg"]*dla["uncorrNeg"]
    end
    uc_pos = sqrt(uc_pos)
    uc_neg = sqrt(uc_neg)
    dl_c /= w
    return Dict("DL"       => dl_c,
                "ucorrPos" => uc_pos,
                "ucorrNeg" => uc_neg,
                "corrPos"  => DL_all(det, "am")["corrPos"],
                "corrNeg"  => DL_all(det, "ba")["corrNeg"])
end

DL_collection = Dict(d => combined_DL(d) for d in detectors)
print("*** DLs calculated")

*** DLs calculated

In [6]:
for d ∈ detectors
    DLs = DL_collection[d]
    printfmt("{1:.2f} +{2:.2f}-{3:.2f}(corr)",DLs["DL"],DLs["corrPos"],DLs["corrNeg"])
    printfmt(" +{1:.2f}-{2:.2f}(uncorr)\n",DLs["ucorrPos"],DLs["ucorrNeg"])
end

0.57 +0.03-0.06(corr) +0.02-0.02(uncorr)
0.80 +0.03-0.06(corr) +0.02-0.02(uncorr)
0.71 +0.03-0.06(corr) +0.02-0.02(uncorr)
0.52 +0.03-0.06(corr) +0.03-0.03(uncorr)
0.61 +0.03-0.04(corr) +0.01-0.01(uncorr)
0.55 +0.03-0.06(corr) +0.06-0.05(uncorr)
0.55 +0.03-0.06(corr) +0.02-0.02(uncorr)
0.71 +0.04-0.05(corr) +0.05-0.04(uncorr)
0.72 +0.03-0.06(corr) +0.04-0.04(uncorr)
0.69 +0.03-0.07(corr) +0.04-0.04(uncorr)
0.86 +0.03-0.07(corr) +0.04-0.03(uncorr)
0.86 +0.03-0.06(corr) +0.03-0.03(uncorr)
0.72 +0.03-0.06(corr) +0.03-0.03(uncorr)
0.84 +0.03-0.06(corr) +0.03-0.03(uncorr)
0.68 +0.03-0.05(corr) +0.04-0.03(uncorr)
0.81 +0.03-0.06(corr) +0.02-0.02(uncorr)
0.69 +0.03-0.07(corr) +0.03-0.03(uncorr)
0.76 +0.03-0.07(corr) +0.03-0.02(uncorr)
0.70 +0.03-0.05(corr) +0.04-0.03(uncorr)
0.67 +0.03-0.06(corr) +0.03-0.03(uncorr)
0.67 +0.03-0.06(corr) +0.04-0.04(uncorr)
0.68 +0.03-0.06(corr) +0.04-0.04(uncorr)
0.63 +0.03-0.05(corr) +0.04-0.03(uncorr)
0.76 +0.03-0.06(corr) +0.04-0.04(uncorr)
0.70 +0.03-0.06(

In [7]:
function calculate_volumes(det)
    vol = volume(det)
    dl          = DL_collection[det]["DL"]
    dl_ucorrPos = DL_collection[det]["ucorrPos"]
    dl_ucorrNeg = DL_collection[det]["ucorrNeg"]
    dl_corrPos  = DL_collection[det]["corrPos"]
    dl_corrNeg  = DL_collection[det]["corrNeg"]
    dl_errPos   = dl_ucorrPos + dl_corrPos
    dl_errNeg   = dl_ucorrNeg + dl_corrNeg
    dl_g        = dl + dl_growth(det)*0.1
    dl_g_err    = dl_growth(det)*0.05
    # add dl growth to correlated uncertainty
    dl_corrPos_g = sqrt(dl_corrPos*dl_corrPos + dl_g_err*dl_g_err)
    dl_corrNeg_g = sqrt(dl_corrNeg*dl_corrNeg + dl_g_err*dl_g_err)
    dl_ucorrPos_g = sqrt(dl_ucorrPos*dl_ucorrPos + dl_g_err*dl_g_err)
    dl_ucorrNeg_g = sqrt(dl_ucorrNeg*dl_ucorrNeg + dl_g_err*dl_g_err)
    dl_totPos_g = sqrt(dl_corrPos_g*dl_corrPos_g + dl_ucorrPos*dl_ucorrPos)
    dl_totNeg_g = sqrt(dl_corrNeg_g*dl_corrNeg_g + dl_ucorrNeg*dl_ucorrNeg)
    
    #printfmt("{1:s} & \\mep{{{2:.2f}}}{{{3:.2f}}}{{{4:.2f}}}{{{5:.2f}}}{{{6:.2f}}}\n",
    #    det, dl_g, dl_corrPos_g, dl_ucorrPos, dl_corrNeg_g, dl_ucorrNeg)
    #printfmt("{1:s} , {2:.2f} , {2:.2f}\n", det, dl_totPos_g, dl_totNeg_g)
    
    # calculate volumes without growth
    av           =      active_volume(det,dl)
    av_errPos    =      active_volume(det,dl-dl_errNeg) - av   # smaller DL bigger AV
    av_errNeg    = av - active_volume(det,dl+dl_errPos)        # bigger DL smaller AV
    av_uncorrPos =      active_volume(det,dl-dl_ucorrNeg) - av # smaller DL bigger AV
    av_uncorrNeg = av - active_volume(det,dl+dl_ucorrPos)      # bigger DL smaller AV
    av_corrPos   = av_errPos - av_uncorrPos
    av_corrNeg   = av_errNeg - av_uncorrNeg
    
    # calculate volumes with growth added
    av_g           =        active_volume(det, dl_g)
    av_g_errPos    =        active_volume(det, dl_g-dl_corrNeg_g-dl_ucorrNeg) - av_g
    av_g_errNeg    = av_g - active_volume(det, dl_g+dl_corrPos_g+dl_ucorrPos)
    av_g_uncorrPos =        active_volume(det, dl_g-dl_ucorrNeg) - av_g
    av_g_uncorrNeg = av_g - active_volume(det, dl_g+dl_ucorrPos)
    av_g_corrPos   = av_g_errPos - av_g_uncorrPos
    av_g_corrNeg   = av_g_errNeg - av_g_uncorrNeg
    
    # calculate asymmetric uncertainties
    # using dl as central value + 0.1mm/yr correlated positive uncertainty
    dl_asymm_corrPos = sqrt(dl_corrPos*dl_corrPos + dl_growth(det)*0.1*dl_growth(det)*0.1)
    av_asymm_errNeg  = av - active_volume(det, dl+dl_asymm_corrPos+dl_ucorrPos)
    av_asymm_corrNeg = av_asymm_errNeg - av_uncorrNeg
    
    return Dict("volume"          => vol,
                "active-volume"   => av,
                "fAV"             => av / vol,
                "fAV-uncorrPos"   => av_uncorrPos / vol, 
                "fAV-uncorrNeg"   => av_uncorrNeg / vol, 
                "fAV-corrPos"     => av_corrPos / vol, 
                "fAV-corrNeg"     => av_corrNeg / vol, 
                "fAV-errPos"      => av_errPos / vol,
                "fAV-errNeg"      => av_errNeg / vol,
        
                "g-active-volume" => av_g,
                "g-fAV"           => av_g / vol,
                "g-fAV-uncorrPos" => av_g_uncorrPos / vol, 
                "g-fAV-uncorrNeg" => av_g_uncorrNeg / vol, 
                "g-fAV-corrPos"   => av_g_corrPos / vol, 
                "g-fAV-corrNeg"   => av_g_corrNeg / vol, 
                "g-fAV-errPos"    => av_g_errPos / vol,
                "g-fAV-errNeg"    => av_g_errNeg / vol,
    
                "a-fAV-corrNeg"   => av_asymm_corrNeg / vol,
                "a-fAV-errNeg"    => av_asymm_errNeg / vol
               )
end

Vc = Dict(d => calculate_volumes(d) for d ∈ detectors)

print("*** Volumes calculated")

GD32A , 0.17 , 0.17
GD32B , 0.13 , 0.13
GD32C , 0.14 , 0.14
GD32D , 0.13 , 0.13
GD35A , 0.17 , 0.17
GD35B , 0.13 , 0.13
GD35C , 0.13 , 0.13
GD61A , 0.16 , 0.16
GD61B , 0.15 , 0.15
GD61C , 0.13 , 0.13
GD76B , 0.15 , 0.15
GD76C , 0.15 , 0.15
GD79B , 0.16 , 0.16
GD79C , 0.13 , 0.13
GD89A , 0.17 , 0.17
GD89B , 0.16 , 0.16
GD89C , 0.16 , 0.16
GD89D , 0.14 , 0.14
GD91A , 0.16 , 0.16
GD91B , 0.15 , 0.15
GD91C , 0.15 , 0.15
GD91D , 0.16 , 0.16
GD00A , 0.15 , 0.15
GD00B , 0.15 , 0.15
GD00C , 0.16 , 0.16
GD00D , 0.15 , 0.15
GD02A , 0.13 , 0.13
GD02B , 0.14 , 0.14
GD02C , 0.15 , 0.15
*** Volumes calculated

## Active volume fraction at the time of measurement

In [8]:
for d ∈ detectors
    printfmt("{1:s} {2:.3f}",d,Vc[d]["fAV"])
    printfmt(" +{1:.3f}-{2:.3f}(corr)",    Vc[d]["fAV-corrPos"],   Vc[d]["fAV-corrNeg"])
    printfmt(" +{1:.3f}-{2:.3f}(uncorr)\n",Vc[d]["fAV-uncorrPos"], Vc[d]["fAV-uncorrNeg"])
end

GD32A 0.923 +0.008-0.004(corr) +0.003-0.003(uncorr)
GD32B 0.910 +0.007-0.003(corr) +0.002-0.002(uncorr)
GD32C 0.922 +0.006-0.003(corr) +0.002-0.002(uncorr)
GD32D 0.941 +0.007-0.003(corr) +0.004-0.004(uncorr)
GD35A 0.933 +0.004-0.003(corr) +0.002-0.002(uncorr)
GD35B 0.939 +0.007-0.003(corr) +0.006-0.007(uncorr)
GD35C 0.931 +0.007-0.004(corr) +0.003-0.003(uncorr)
GD61A 0.920 +0.005-0.004(corr) +0.004-0.006(uncorr)
GD61B 0.918 +0.007-0.003(corr) +0.005-0.005(uncorr)
GD61C 0.915 +0.008-0.004(corr) +0.005-0.005(uncorr)
GD76B 0.884 +0.009-0.004(corr) +0.004-0.005(uncorr)
GD76C 0.908 +0.006-0.003(corr) +0.003-0.003(uncorr)
GD79B 0.916 +0.007-0.003(corr) +0.004-0.004(uncorr)
GD79C 0.906 +0.007-0.003(corr) +0.003-0.003(uncorr)
GD89A 0.912 +0.006-0.004(corr) +0.004-0.005(uncorr)
GD89B 0.897 +0.007-0.004(corr) +0.003-0.003(uncorr)
GD89C 0.912 +0.009-0.004(corr) +0.004-0.004(uncorr)
GD89D 0.897 +0.009-0.004(corr) +0.003-0.004(uncorr)
GD91A 0.917 +0.006-0.003(corr) +0.004-0.005(uncorr)
GD91B 0.922 

## Active volume fration with growth added

In [9]:
for d ∈ detectors
    printfmt("{1:s} {2:.3f}",d,Vc[d]["g-fAV"])
    printfmt(" +{1:.3f}-{2:.3f}(corr)",    Vc[d]["g-fAV-corrPos"],   Vc[d]["g-fAV-corrNeg"])
    printfmt(" +{1:.3f}-{2:.3f}(uncorr)\n",Vc[d]["g-fAV-uncorrPos"], Vc[d]["g-fAV-uncorrNeg"])
end

GD32A 0.880 +0.023-0.022(corr) +0.003-0.003(uncorr)
GD32B 0.883 +0.015-0.014(corr) +0.002-0.002(uncorr)
GD32C 0.894 +0.015-0.014(corr) +0.002-0.002(uncorr)
GD32D 0.914 +0.015-0.014(corr) +0.003-0.003(uncorr)
GD35A 0.897 +0.018-0.018(corr) +0.001-0.001(uncorr)
GD35B 0.915 +0.014-0.013(corr) +0.005-0.006(uncorr)
GD35C 0.902 +0.016-0.015(corr) +0.003-0.003(uncorr)
GD61A 0.889 +0.016-0.016(corr) +0.004-0.005(uncorr)
GD61B 0.888 +0.017-0.015(corr) +0.004-0.004(uncorr)
GD61C 0.887 +0.017-0.014(corr) +0.005-0.005(uncorr)
GD76B 0.848 +0.020-0.018(corr) +0.004-0.005(uncorr)
GD76C 0.878 +0.016-0.015(corr) +0.003-0.003(uncorr)
GD79B 0.882 +0.018-0.017(corr) +0.003-0.003(uncorr)
GD79C 0.879 +0.015-0.014(corr) +0.003-0.003(uncorr)
GD89A 0.872 +0.021-0.020(corr) +0.004-0.005(uncorr)
GD89B 0.860 +0.020-0.019(corr) +0.003-0.003(uncorr)
GD89C 0.875 +0.020-0.018(corr) +0.004-0.004(uncorr)
GD89D 0.863 +0.020-0.017(corr) +0.003-0.004(uncorr)
GD91A 0.883 +0.018-0.017(corr) +0.004-0.005(uncorr)
GD91B 0.890 

## Active volume fraction as measured with growth added as asymmetric uncertainty

In [10]:
for d ∈ detectors
    printfmt("{1:s} {2:.3f}",d,Vc[d]["fAV"])
    printfmt(" +{1:.3f}-{2:.3f}(corr)",    Vc[d]["fAV-corrPos"],   Vc[d]["a-fAV-corrNeg"])
    printfmt(" +{1:.3f}-{2:.3f}(uncorr)\n",Vc[d]["fAV-uncorrPos"], Vc[d]["fAV-uncorrNeg"])
end

GD32A 0.923 +0.008-0.044(corr) +0.003-0.003(uncorr)
GD32B 0.910 +0.007-0.027(corr) +0.002-0.002(uncorr)
GD32C 0.922 +0.006-0.028(corr) +0.002-0.002(uncorr)
GD32D 0.941 +0.007-0.028(corr) +0.004-0.004(uncorr)
GD35A 0.933 +0.004-0.036(corr) +0.002-0.002(uncorr)
GD35B 0.939 +0.007-0.025(corr) +0.006-0.007(uncorr)
GD35C 0.931 +0.007-0.029(corr) +0.003-0.003(uncorr)
GD61A 0.920 +0.005-0.031(corr) +0.004-0.006(uncorr)
GD61B 0.918 +0.007-0.031(corr) +0.005-0.005(uncorr)
GD61C 0.915 +0.008-0.029(corr) +0.005-0.005(uncorr)
GD76B 0.884 +0.009-0.036(corr) +0.004-0.005(uncorr)
GD76C 0.908 +0.006-0.030(corr) +0.003-0.003(uncorr)
GD79B 0.916 +0.007-0.035(corr) +0.004-0.004(uncorr)
GD79C 0.906 +0.007-0.027(corr) +0.003-0.003(uncorr)
GD89A 0.912 +0.006-0.040(corr) +0.004-0.005(uncorr)
GD89B 0.897 +0.007-0.038(corr) +0.003-0.003(uncorr)
GD89C 0.912 +0.009-0.037(corr) +0.004-0.004(uncorr)
GD89D 0.897 +0.009-0.035(corr) +0.003-0.004(uncorr)
GD91A 0.917 +0.006-0.034(corr) +0.004-0.005(uncorr)
GD91B 0.922 